In [2]:
import os
import pandas as pd
import numpy as np
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [2]:
load_dotenv(override=True)

True

In [3]:
os.getcwd()

'C:\\Users\\Prithvi\\Downloads\\Practice\\Udemy - LLM Engineering\\llm_engineering\\week1'

In [1]:
api_key = os.getenv('OPENAI_API_KEY')

NameError: name 'os' is not defined

In [ ]:
api_key

#### Check the current enviroment

In [5]:
import sys
sys.executable

'C:\\Users\\Prithvi\\anaconda3\\envs\\udemyllm\\python.exe'

In [6]:
os.environ.get('CONDA_DEFAULT_ENV')

'udemyllm'

#### Check API key correctness

In [7]:
if api_key.startswith('sk-proj') and len(api_key) > 10:
    print("API key looks correct")
else:
    print("API key is not correct")

API key looks correct


In [219]:
MODEL = 'tinyllama:latest'
MODEL = 'gemma3:latest'
MODEL = 'gpt-4.1-nano'

In [220]:
# openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
openai = OpenAI()

In [221]:
openai

In [222]:
type(openai)

openai.OpenAI

In [223]:
MODEL

'gpt-4.1-nano'

In [203]:
headers= {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [204]:
type(headers)

dict

In [205]:
url = "https://www.bbc.co.uk/"

In [138]:
response = requests.get(url, headers=headers)

In [140]:
body = response.content

In [142]:
type(body)

bytes

In [143]:
soup = BeautifulSoup(body, 'html.parser')

In [147]:
for i in soup.body(["script", "style", "img", "input"]):
    i.decompose()

In [149]:
text = soup.body.get_text(separator = "\n", strip = True)

In [158]:
links = [i.get('href') for i in soup.find_all("a")]

In [160]:
type(links)

list

In [161]:
url

'https://www.bbc.co.uk/'

In [165]:
a= f"Input website is: {url}"

In [166]:
a

'Input website is: https://www.bbc.co.uk/'

In [167]:
type(a)

str

In [224]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [126]:
a = Website("https://www.bbc.co.uk/")

In [ ]:
f"Webpage Title

In [175]:
a = "how are you doing \
I am doing fine"

In [176]:
a

'how are you doing I am doing fine'

In [177]:
print(a)

how are you doing I am doing fine


In [127]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [128]:
link_system_prompt

'You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.\nYou should respond in JSON as in this example:\n{\n    "links": [\n        {"type": "about page", "url": "https://full.url/goes/here/about"},\n        {"type": "careers page": "url": "https://another.full.url/careers"}\n    ]\n}\n'

In [178]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [206]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [225]:
# MODEL = 'gemma3:latest'

In [226]:
url

'https://www.bbc.co.uk/'

In [227]:
a = Website(url)

In [300]:
a = openai.chat.completions.create(model = MODEL,
                              messages = [
                                  {"role": "system", 
                                  "content": "Answer in markdown"},
                                  {"role": "user",
                                  "content": "What is capital of France"}
                              ]
                              )

In [305]:
a

ChatCompletion(id='chatcmpl-Bv9MGGiliSUWQaw2qdXBaqid6hBv6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1752960368, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=19, total_tokens=26, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [313]:
a.choices[0].message.content

'The capital of France is Paris.'

In [314]:
MODEL

'gpt-4.1-nano'

In [316]:
d1 = {
    "name": "Alice",
    "age": 30,
    "is_student": False
}

In [317]:
d1

{'name': 'Alice', 'age': 30, 'is_student': False}

In [318]:
type(d1)

dict

In [320]:
for i in d1.items():
    print(i)

('name', 'Alice')
('age', 30)
('is_student', False)


In [322]:
j1 = json.dumps(d1)

In [323]:
j1

'{"name": "Alice", "age": 30, "is_student": false}'

In [324]:
type(j1)

str

In [325]:
print(j1)

{"name": "Alice", "age": 30, "is_student": false}


In [326]:
d1.keys()

dict_keys(['name', 'age', 'is_student'])

In [327]:
d1.values()

dict_values(['Alice', 30, False])

In [329]:
for i, j in enumerate(d1.items()):
    print(i, j)

0 ('name', 'Alice')
1 ('age', 30)
2 ('is_student', False)


In [330]:
d1['name']

'Alice'

In [331]:
d1.keys()

dict_keys(['name', 'age', 'is_student'])

In [334]:
k1 = json.loads(j1)

In [335]:
k1

{'name': 'Alice', 'age': 30, 'is_student': False}

In [336]:
type(k1)

dict

In [229]:
get_links("https://www.bbc.co.uk/")

{'links': [{'type': 'about page', 'url': 'https://www.bbc.co.uk/aboutthebbc'},
  {'type': 'governance/mission',
   'url': 'https://aboutthebbc/governance/mission'},
  {'type': 'what we do', 'url': 'https://www.bbc.co.uk/whatwedo/'},
  {'type': 'documents',
   'url': 'https://www.bbc.co.uk/documents/bbcmsa2025.pdf'}]}

In [230]:
url

'https://www.bbc.co.uk/'

In [231]:
url = "https://edwarddonner.com"

In [232]:
url

'https://edwarddonner.com'

In [233]:
get_links(url)

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'linkedin profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [234]:
MODEL

'gpt-4.1-nano'

In [235]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")

In [236]:
huggingface

In [240]:
huggingface.title

'Hugging Face – The AI community building the future.'

In [241]:
huggingface.url

'https://huggingface.co'

In [243]:
a = Website("https://timesofindia.indiatimes.com/")

In [245]:
a.title

'News - Breaking News,  Latest News, India News, World News, Bollywood, Sports, Business and Political News | Times of India'

In [246]:
url = "https://huggingface.co/"

In [247]:
get_links(url)

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/docs'},
  {'type': 'company page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'press/brand info', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://blog.huggingface.co/'}]}

#### Create brochure

In [248]:
url

'https://huggingface.co/'

In [250]:
a = Website(url)

In [258]:
result = "Landing page: \n"

In [259]:
result

'Landing page: \n'

In [260]:
print(result)

Landing page: 



In [261]:
a.title

'Hugging Face – The AI community building the future.'

In [262]:
a.url

'https://huggingface.co/'

In [265]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [341]:
url = "https://www.anthropic.com/"

In [342]:
a = get_all_details(url)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'solutions pages', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions pages', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions pages', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'solutions pages', 'url': 'https://www.anthropic.com/solutions/education'}, {'type': 'solutions pages', 'url': 'https://www.anthropic.com/solutions/financial-services'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'company info', 'url': 'https://www.anthropic.com/company'}, {'type': 'investor or partnership info', 'url': 'https://www.anthrop

In [266]:
b = get_all_details(url)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'company page', 'url': 'https://huggingface.co'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter profile', 'url': 'h

In [268]:
type(b)

str

In [285]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Make it humorous"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [286]:
print(system_prompt)

You are an assistant that analyzes the contents of several relevant pages from a company website and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.Include details of company culture, customers and careers/jobs if you have the information. Make it humorous


In [287]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [292]:
a = get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/docs'}, {'type': 'company page', 'url': 'https://huggingface.co'}, {'type': 'models overview', 'url': 'https://huggingface.co/models'}, {'type': 'datasets overview', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces overview', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

In [289]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [345]:
url

'https://www.anthropic.com/'

In [359]:
stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt("Anthropic", url)}
          ],
        stream=True
    )

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/customer-support'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/education'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/financial-services'}, {'type': 'research', 'url': 'https://www.anthropic.com/research'}, {'type': 'partners', 'url': 'https://www.anthropic.com/partners/mcp'}, {'type': 'partners', 'url': 'https://www.anthropic.com/partners/powered-by-claude'}]}


In [360]:
for i in stream:
    print(i.choices[0].delta.content, end = "")

# Welcome to Anthropic: The AI Company with a Sense of Humor (and Responsibility!)

## Who Are We?

At **Anthropic**, we're building the future of AI—think less "Skynet" and more "Helpful Sidekick." Our mission? Create *powerful*, *safe*, and *humane* AI that benefits humankind — because just because machines can think faster than us doesn’t mean they should outsmart us on kindness.

## Our Big Brain (Models)

- **Claude Opus 4**: Our flagship, the Einstein of AI models. It’s so smart it makes Jeff Goldblum look like a beginner.
- **Claude Sonnet 4**: Our poetic genius—yes, AI poetry is a thing now.
- **Claude Haiku 3.5**: Perfect for those quick, Zen moments of enlightenment... or simply debugging code.

## What Can You Do With Us?

- Build **AI apps** with our APIs (think: robot that actually understands what you’re saying).
- Deploy **AI agents** that collaborate, code, support, educate, and even (wait for it) *think responsibly*.
- Use our models for **financial services** or **cus

In [290]:
url

'https://huggingface.co/'

In [293]:
url = "https://edwarddonner.com"

In [294]:
url

'https://edwarddonner.com'

In [296]:
# stream_brochure("HuggingFace", url)
stream_brochure("Edwarddoner", url)

Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'blog or news', 'url': 'https://edwarddonner.com/posts/'}, {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


# Welcome to Edwarddoner: Where AI Battles and Human Dreams Collide (In the Most Delightfully Nerdy Way)

---

## About Us: Turning Code into (Potentially) World-Changing Magic

At Edwarddoner, we don’t just dabble in AI—we throw ourselves headfirst into the digital *arena* of diplomacy, devious tactics, and… sometimes, just plain old cleverness. Think of it as "Game of Thrones" but with LLMs instead of dragons, and maybe a little less blood and gore (but only marginally).

Our hero, Ed—co-founder & CTO of Nebula.io—has a passion for writing code, DJing badly but enthusiastically, and exploring the wild frontiers of AI in human resources. Because nothing says “fun weekend” like helping people find their dream jobs, right?

---

## The Battle Arena: Connect Four, but Make It AI

Our Connect Four-esque platform doesn’t just sit there gathering dust. We pit Large Language Models (LLMs) against each other in a fierce competition of diplomacy, deception, and digital wit. Imagine chess, but with more data and possibly less etiquette.

---

## Why Choose Us? Because Robots Should Be Smarter Than Your Average Matchmaker

Our proprietary, patented matching model is so good, it doesn’t even need keywords—beat that, traditional recruiters! We work with ground-breaking LLMs designed specifically for talent, helping recruiters source, understand, engage, and manage talent faster than you can say “Hadoop.”

And yes, we’re winners—award-winning, press-covered, and loved by happy customers. The workplace of tomorrow, today.

---

## Company Culture: Nerdy, Noble, and Slightly Quirky

- **Innovators & Experimenters:** Ed’s motto? “Why not try that thing nobody asked for?” From electronic music to AI experiments, we thrive on curiosity.
- **Passionate Problem-Solvers:** Our roots trace back to untapt, where we tackled real-world problems like tech talent shortages and got featured in Fast Company and Forbes. Our mission? Helping people discover their *Ikigai*—that sweet spot where purpose and prosperity intersect.
- **Humorous & Human:** We’re serious about AI but don’t take ourselves too seriously. Expect quirky coffee chats, spontaneous debates on who's the better DJ, and the occasional meme about machine learning.

---

## Careers & Jobs: Join the AI Revolution (No Tuxedo Required)

Looking to put your talent into *really* impactful work? We’re hiring! Whether you’re a coding wizard, a data science maestro, or someone who just wants to tell their grandma they’re “working in AI,” Edwarddoner has a spot for you.

Join us to help reshape the workforce, one line of code at a time. And if you’re worried about hand-eye coordination—don’t be. Our success is powered by brains, not hand-eye skills.

---

## Connect With Us!

- Email: ed [at] edwarddonner [dot] com
- Follow Ed on [LinkedIn](#), [Twitter](#), or [Facebook](#)
- Subscribe to our newsletter—don’t worry, we promise not to spam you, just some good ol’ AI banter!

---

## Why Wait?

Come for the AI battles, stay for the meaningful impact, and leave with a sense that maybe, just maybe, the robots aren’t here to take over—but to help us all find our *Ikigai*.

*Edwarddoner—where ambition meets algorithm.*